# Chat vs Completion Model

In the context of OpenAI's GPT-3.5 architecture ""chat" and "completion" refer to two different ways of interacting with the model, and they serve different purposes:

1. **Chat Model:**
   - The "chat" format is designed for multi-turn conversations or dialogues.
   - It is typically used when you want to have back-and-forth interactions with the model.
   - You provide a series of messages as input, where each message has a role ("system," "user," or "assistant") and content (the text of the message). The conversation usually starts with a system message to set the context, followed by alternating user and assistant messages.
   - The model processes the entire conversation history to generate a response, taking into account the context provided in previous messages.
   - Chat models are useful for creating chatbots, virtual assistants, or simulating conversation with a knowledgeable entity.
   

2. **Completion Model:**
   - The "completion" format is designed for single-turn tasks or generating text based on a prompt.
   - It is typically used for tasks like language translation, text summarization, text generation, and more.
   - You provide a single prompt as input, and the model generates a completion or response based on that prompt.
   - Completion models are useful for generating text that doesn't require ongoing interaction or conversation context.

In summary, the main difference between the two lies in the format of input and the use case. "Chat" is used for multi-turn conversations, while "completion" is used for single-turn tasks or text generation based on a prompt. Keep in mind that OpenAI's models and offerings may have evolved since my last update in September 2021, so I recommend checking OpenAI's official documentation for the most up-to-date information on their offerings and capabilities.

In [12]:
import yaml, os, openai
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, FewShotPromptTemplate

In [ ]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['ENGINE'] = credentials['ENGINE']

openai.api_key = credentials['OPENAI_API_KEY']
openai.api_base = credentials['OPENAI_API_BASE']
openai.api_type = credentials['OPENAI_API_TYPE']
openai.api_version = credentials['OPENAI_API_VERSION']
openai.engine = credentials['ENGINE']

In [14]:
response = openai.ChatCompletion.create(
                                        engine = os.environ["ENGINE"],
                                        model="gpt-3.5-turbo",
                                        messages=[
                                                {
                                                "role": "system", 
                                                "content": "You are a helpful assistant."
                                                },
                                                {
                                                "role": "user", 
                                                "content": "Hello what kind of assistant are you?"
                                                },
                                            ]
                                        )

In [15]:
response

<OpenAIObject chat.completion id=chatcmpl-85GANtEaUrrfLHgp74qdDu6CWdVSo at 0x125ebcf90> JSON: {
  "id": "chatcmpl-85GANtEaUrrfLHgp74qdDu6CWdVSo",
  "object": "chat.completion",
  "created": 1696264475,
  "model": "gpt-35-turbo-16k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Hello! I am a helpful assistant designed to assist with a wide range of tasks and provide information and support. I can answer questions, provide recommendations, assist with organization and planning, and much more. How can I assist you today?"
      }
    }
  ],
  "usage": {
    "completion_tokens": 47,
    "prompt_tokens": 25,
    "total_tokens": 72
  }
}

### Chat Markup Language - token system

```markdown
<|im_start|>system
You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.
Knowledge cutoff: 2021-09-01
Current date: 2023-03-01<|im_end|>
<|im_start|>user
How are you<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
How are you now?<|im_end|>
```

```
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```

In [16]:
messages=[
        {"role": "system", "content": "You are a helpful assistant named Kate."},
        {"role": "user", "content": "Hello what kind of assistant are you?"},
    ]

In [17]:
conversation_total_tokens = 0

while True:
    message = input("Human: ")
    if message=='exit':
        print(f"{conversation_total_tokens} tokens used in total in this conversation")
        break
    if message:
        messages.append(
                        {
                            "role": "user", 
                            "content": message
                            },
                        )
        response = openai.ChatCompletion.create(
                                                engine = os.environ["ENGINE"],
                                                model="gpt-3.5-turbo", 
                                                messages=messages
                                                )
    
    reply = response.choices[0].message.content
    total_tokens = response.usage['total_tokens']
    conversation_total_tokens += total_tokens
    print(f"ChatGPT: {reply} \n {total_tokens} tokens used")
    messages.append({"role": "assistant", "content": reply})

ChatGPT: Hello! I'm Kate, your helpful assistant. How can I assist you today? 
 49 tokens used
ChatGPT: I am a virtual assistant designed to help with a wide range of tasks and provide information on various topics. I can assist with answering questions, providing recommendations, giving directions, setting reminders, and much more. How can I assist you today? 
 112 tokens used
ChatGPT: Yes, I have knowledge about football. Football, also known as soccer in some parts of the world, is a popular sport played by teams of 11 players each. It is played on a rectangular field where the objective is to score a goal by kicking the ball into the opposing team's net. The sport is governed by various organizations, such as FIFA (Fédération Internationale de Football Association), and has a rich history and culture surrounding it. If you have any specific questions about football, feel free to ask! 
 231 tokens used
ChatGPT: As an AI assistant, I don't have personal opinions or biases. The debate

## OpenAI ChatGPT - Language Chain Integration
### But why ?
        - because we can change the output by employing control mechanisms via prompt template and can dynamically change the output

In [20]:
prefix_messages = [{
                    "role": "system", 
                    "content": "You are a helpful history professor named Zuu."
                    }]


In [21]:
llm = ChatOpenAI(
                openai_api_key=os.environ["OPENAI_API_KEY"],
                engine = os.environ["ENGINE"],
                model='gpt-3.5-turbo',
                temperature=0.0, 
                max_tokens = 256
                )

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [24]:
# prompt_template = """Take the following question: {user_input}

# Answer it in an informative and intersting but conscise way for someone who is new to this topic."""

prompt_template = """Take the following question: {user_input}

Answer should be very short"""

prompt = PromptTemplate(
                        template=prompt_template, 
                        input_variables=["user_input"]
                        )

#### here we employed that the response should be "informative and intersting but conscise"

In [25]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
user_input = "Which is better ? barcelona or real madrid"
llm_chain.run(user_input)

'It is subjective and depends on personal preference.'